In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('city_payroll_data.csv')

C:\Users\chris\AppData\Local\Temp\ipykernel_24224\3698127440.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('city_payroll_data.csv')


In [3]:
ed_agencies = [
# 'DEPT OF ED PER DIEM TEACHERS',
# 'DEPT OF ED PER SESSION TEACHER',
# 'DEPARTMENT OF EDUCATION ADMIN',
# 'DEPT OF ED HRLY SUPPORT STAFF',
# 'DEPT OF ED PARA PROFESSIONALS',
'DEPT OF ED PEDAGOGICAL',
# 'TEACHERS RETIREMENT SYSTEM',
# 'NYC EMPLOYEES RETIREMENT SYS'
]

In [5]:
df = data[(data['Agency Name']=='DEPT OF ED PEDAGOGICAL')\
          &(data['Title Description']=='TEACHER')]

In [6]:
df = df.sort_values(by='Fiscal Year')
df = df.reset_index(drop=True)

In [7]:
df.head()

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2014,742.0,DEPT OF ED PEDAGOGICAL,MOREL,LAURA,NaN,09/06/2011,NaN,TEACHER,ACTIVE,60626.0,per Annum,0.0,60620.03,0.0,0.0,1000.0
1,2014,742.0,DEPT OF ED PEDAGOGICAL,DELGADO,LILLIAN,M,09/08/1998,NaN,TEACHER,ACTIVE,79531.0,per Annum,0.0,78791.12,0.0,0.0,1000.0
2,2014,742.0,DEPT OF ED PEDAGOGICAL,DEGRANGE,TRICIA,A,09/08/1998,NaN,TEACHER,ACTIVE,79531.0,per Annum,0.0,77385.43,0.0,0.0,1000.0
3,2014,742.0,DEPT OF ED PEDAGOGICAL,DEGRAFFENREID,ARLEEN,NaN,09/08/1998,NaN,TEACHER,ACTIVE,79531.0,per Annum,0.0,78791.12,0.0,0.0,1000.0
4,2014,742.0,DEPT OF ED PEDAGOGICAL,DEEGAN,TIMOTHY,F,09/08/1998,NaN,TEACHER,ACTIVE,79531.0,per Annum,0.0,79073.44,0.0,0.0,1000.0


In [8]:
df['Agency Start Date'] = pd.to_datetime(df['Agency Start Date'], errors='coerce')

In [9]:
df['Work Location Borough'] = df['Work Location Borough'].str.strip().str.title()

In [10]:
df = df.drop(columns=['Payroll Number', 'Agency Name',
                 'Title Description', 'Pay Basis',
                 'Regular Hours', 'OT Hours', 'Total OT Paid'])

In [11]:
df['Total Other Pay'].describe()

count    617422.000000
mean       2173.025292
std        4512.517886
min      -64984.350000
25%           0.000000
50%           5.200000
75%        2389.257500
max      149273.710000
Name: Total Other Pay, dtype: float64

In [12]:
df[df['Regular Gross Paid'] < 0].count()

Fiscal Year                   620
Last Name                     620
First Name                    620
Mid Init                      415
Agency Start Date             620
Work Location Borough         619
Leave Status as of June 30    620
Base Salary                   620
Regular Gross Paid            620
Total Other Pay               620
dtype: int64

In [13]:
df[df['Total Other Pay'] < 0].count()

Fiscal Year                   12116
Last Name                     12116
First Name                    12116
Mid Init                       7198
Agency Start Date             12114
Work Location Borough         11900
Leave Status as of June 30    12116
Base Salary                   12116
Regular Gross Paid            12116
Total Other Pay               12116
dtype: int64

In [14]:
df['Total Pay'] = df[] + df['Total Other Pay']

SyntaxError: invalid syntax (4109251419.py, line 1)